In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk', 'koi_slogg']]
X = selected_features
X.head()

,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_slogg
0,54.418383,2.479000e-04,-2.479000e-04,162.513840,4.467
1,19.899140,1.490000e-05,-1.490000e-05,175.850252,4.544
2,1.736952,2.630000e-07,-2.630000e-07,170.307565,4.564
3,2.525592,3.760000e-06,-3.760000e-06,171.595550,4.438
4,4.134435,1.050000e-05,-1.050000e-05,172.979370,4.486


# Create a Train Test Split

Use `koi_disposition` for the y values

In [9]:
from sklearn.model_selection import train_test_split

y = df["koi_disposition"]
y = pd.get_dummies(y)
# y.head()


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [8]:
X_train.head()

,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_slogg
6122,6.768901,7.380000e-05,-7.380000e-05,133.077240,4.327
6370,0.733726,6.060000e-06,-6.060000e-06,132.020050,4.578
2879,7.652707,6.540000e-05,-6.540000e-05,134.460380,4.481
107,7.953547,1.910000e-05,-1.910000e-05,174.662240,4.536
29,4.959319,5.150000e-07,-5.150000e-07,172.258529,4.359


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [11]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [13]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)

LinearRegression()

# Train the Model



In [14]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: -0.5708106209944682
Testing Data Score: -0.5590364804516611


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [20]:
# Create the SVC Model
from sklearn.svm import SVC 
model2 = SVC(kernel='linear')
model2

SVC(kernel='linear')

In [24]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid2 = GridSearchCV(model2, param_grid, verbose=3)

In [25]:
# Train the model with GridSearch
grid2.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................
[CV] ..................... C=1, gamma=0.0001, score=nan, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ..................... C=1, gamma=0.0001, score=nan, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ..................... C=1, gamma=0.0001, score=nan, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ..................... C=1, gamma=0.0001, score=nan, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ..................... C=1, gamma=0.0001, score=nan, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ..................... C=1, gamma=0.0005, score=nan, total=   0.0s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py", line 162, in fit
    accept_large_sparse=False)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    return f(**kwargs)
  File "C:\Users\woodi\anaconda

C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py", line 162, in fit
    accept_large_sparse=False)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    return f(**kwargs)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py", line 808, in ch

[CV] ..................... C=5, gamma=0.0005, score=nan, total=   0.0s
[CV] C=5, gamma=0.0005 ...............................................
[CV] ..................... C=5, gamma=0.0005, score=nan, total=   0.0s
[CV] C=5, gamma=0.0005 ...............................................
[CV] ..................... C=5, gamma=0.0005, score=nan, total=   0.0s
[CV] C=5, gamma=0.0005 ...............................................
[CV] ..................... C=5, gamma=0.0005, score=nan, total=   0.0s
[CV] C=5, gamma=0.001 ................................................
[CV] ...................... C=5, gamma=0.001, score=nan, total=   0.0s
[CV] C=5, gamma=0.001 ................................................
[CV] ...................... C=5, gamma=0.001, score=nan, total=   0.0s
[CV] C=5, gamma=0.001 ................................................
[CV] ...................... C=5, gamma=0.001, score=nan, total=   0.0s
[CV] C=5, gamma=0.001 ................................................
[CV] .

C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py", line 162, in fit
    accept_large_sparse=False)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    return f(**kwargs)
  File "C:\Users\woodi\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py", line 808, in ch

ValueError: y should be a 1d array, got an array of shape (5243, 3) instead.

In [27]:
print(grid2.best_params_)


{'C': 1, 'gamma': 0.0001}


In [28]:
print(grid2.best_score_)

nan


# Save the Model

In [31]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'burnettawood.csv'
joblib.dump(model2, filename)

['burnettawood.csv']